In [26]:
from goblin_lite.goblin import ScenarioRunner
from goblin_lite.resource_manager.data_fetcher import DataFetcher
import sys
import os
import pandas as pd
import json
import yaml


## Paths and Input variables

We need an input and output path, and emissions factor country key (added so that we might add additional countries in the future), 
baseline year (the reference year for the data, usually 2020, beyond this, there will be an error as not all dependent databases contain data beyond 2020), the target year (usually 2050). 

The configuration files taken as inputs is the scenario config and the forest_config

#### Notes: 
testing outside of these year ranges has not been heavily done. You can try and put 2024 as the reference year for example, but
it will ungracefully return an error. 

In [27]:
input_path ="../data/input"
output_path = "../data/output"

ef_country = "ireland" #keep lower case, I have not implemented checks for this yet

baseline_year = 2020
target_year = 2050

scenario_config = os.path.join(input_path, "scenario_set_0.json")
forest_config = os.path.join(input_path, "forest_config", "cbm_factory.yaml")

### Scenario input files 

Before defining the ScenarioRunner, lets look at the inputs and what they are used for: 

In [28]:
#scenario_config file, defines values for user inputs

with open(scenario_config, 'r') as f:
    scenario_data = json.load(f)

# Print JSON data
print("Scenario Config Data:")
print(json.dumps(scenario_data, indent=4))

Scenario Config Data:
[
    {
        "Scenarios": 0,
        "Manure management cattle": "tank liquid",
        "Manure management sheep": "solid",
        "Dairy pop": 1374930.7168730246,
        "Beef pop": 333997.84938194766,
        "Upland sheep pop": 499320.0,
        "Lowland sheep pop": 1997280.0,
        "Dairy prod": 0.1767579594474441,
        "Beef prod": 0.7522453267393894,
        "Lowland sheep prod": 0,
        "Upland sheep prod": 0,
        "Forest area": 0.34956730774276357,
        "Conifer proportion": 0.24775467326061063,
        "Broadleaf proportion": 0.7522453267393894,
        "Conifer harvest": 0.09594122774517429,
        "Conifer thinned": 0.09594122774517429,
        "Broadleaf harvest": 0,
        "Crop area": 0,
        "Wetland area": 0.6504326922572364,
        "Dairy GUE": 0.017675795944744412,
        "Beef GUE": 0.027484170038358804,
        "Dairy Pasture fertilisation": 147.97061387258714,
        "Beef Pasture fertilisation": 132.65717287175727,

The majority of the values are floats represeting proportions (0:1) for different inputs. Others, like pop, are still floats, but are animal populations. The Scenario variable, is an int, with the number of that particular scenario. 

Afforest year, is the year afforestation ends. Spared area (will explain in person what that is) can be used by different ladn uses, when it is afforested, it is afforested over a number of years. It can be spread longer than the target year, but the minimum is the target year +1 

In [10]:
# Forest config file, defines values for forest inputs
with open(forest_config, 'r') as f:
    forest_data = yaml.safe_load(f)

# Print JSON data
print("Forest Config Data:")
print(yaml.dump(forest_data, indent=4))

Forest Config Data:
Classifiers:
    age_classes:
        age_interval: 5
        max_age: 100
    harvest:
        clearfell:
        -   conifer: 0.5
        -   broadleaf: 0.06
        thinning:
        -   conifer: 0.4
        -   broadleaf: 0
    species:
    - Sitka
    - SGB
    yield_class:
        SGB:
        -   YC10: 1
        Sitka:
        -   YC13_16: 0.37
        -   YC17_20: 0.26
        -   YC20_24: 0.2
        -   YC24_30: 0.17



The configuration here sets up classifiers for the carbon sequestration model for forest (CBM CSF3). This allows us to defined parameters for previously planted standing forest into the future. Basically, what we assume is happening to prexisting forest when we run out of data for it. 

The yield_class variable is only really used for GeoGOBLIN, where we do not have any data on the yield class bins for forest in the catchments. So, we take the forest area, and we apply national distributions based on the National Forest Inventory. 

For now, the only important factors are clearfell and thinning. 

### Scenario Runner

Here we will define the scenario runner, run the simulations. 

In [21]:
runner = ScenarioRunner(ef_country, baseline_year, target_year, scenario_config, forest_config)

In [22]:
# run scenarios 
runner.run_scenarios()

No tables to clean.
JSON file detected


/home/colmduff/Dropbox/projects/FORESIGHT/ICHEC/goblin_resources/.venv/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-5.15.0-116-generic-x86_64-with-glibc2.35
  warnings.warn(message, RuntimeWarning)
/home/colmduff/Dropbox/projects/FORESIGHT/ICHEC/goblin_resources/.venv/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-5.15.0-116-generic-x86_64-with-glibc2.35
  warnings.warn(message, RuntimeWarning)
/home/colmduff/Dropbox/projects/FORESIGHT/ICHEC/goblin_resources/.venv/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-5.15.0-116-generic-x86_64-with-glibc2.35
  warnings.warn(message, RuntimeWarning)
/home/colmduff/Dropbox/projects/FORESIGHT/ICHEC/goblin_resources/.venv/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-5.15.0-116-gen

Scenario Generation Complete ... 



### Fetching Data 

We use the fetcher class to retrieve the data saved locally in the package db

In [23]:
fetcher_class = DataFetcher()

In [24]:
# pull data from the database

aggregated_emmissions_df = fetcher_class.get_climate_change_emission_totals()

print(aggregated_emmissions_df)

              CH4        N2O           CO2          CO2e
index                                                   
-1     634.471263  23.086912    927.628957  24810.855928
 0     496.559922  17.312377 -11188.790227   7302.667634
 1     428.139824  14.847373  -3869.398042  12053.070785
 2     502.186011  16.990445  -8402.569932  10161.106262
 3     405.981400  14.972725 -11153.409297   4181.842125
 4     427.406159  14.365797  -6373.162293   9401.146274


There are over 28 tables returned from a sceanrio run ... 

In [25]:
#Land use emissions by category 

land_emissions_df = fetcher_class.get_landuse_emissions_totals()

print(land_emissions_df)


           land_use  year           CO2         CH4       N2O          CO2e
scenario                                                                   
-1         cropland  2020      0.000000    0.000154  0.000004      0.005380
-1        grassland  2020   1061.271545   42.841175  0.001507   2261.223915
-1           forest  2020  -1496.333122    1.926434  0.512809  -1306.498711
-1          wetland  2020    853.805160   81.496830  0.101991   3162.744029
-1            total  2020    418.743583  126.264594  0.616311   4117.474613
 0         cropland  2050      0.000000    0.000154  0.000004      0.005380
 0        grassland  2050    341.057835   55.223805  0.000725   1887.516399
 0           forest  2050 -12877.875589    1.926434  0.512809 -12688.041178
 0          wetland  2050    853.805160   81.496830  0.101991   3162.744029
 0            total  2050 -11683.012594  138.647224  0.615528  -7637.775370
 1         cropland  2050      0.000000    0.000154  0.000004      0.005380
 1        gr